In [1]:
import numpy as np
import yaml
from onsager import OnsagerCalc
from onsager import crystal
from onsager import crystalStars as stars
from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
def build_point_group_ops(lattice, basis, threshold=1e-8):

        pgroup_ops = []
        sgroup_ops = []
        inv_lat_vec = np.linalg.inv(lattice)
        supercellvect = [np.array((n0, n1, n2))
                         for n0 in range(-1, 2)
                         for n1 in range(-1, 2)
                         for n2 in range(-1, 2)
                         if (n0, n1, n2) != (0, 0, 0)]
        nmat_list = [X for X in [np.array((n0, n1, n2))
                                 for n0 in supercellvect
                                 for n1 in supercellvect
                                 for n2 in supercellvect] if abs(np.linalg.det(X)) == 1]
        for nmat in nmat_list:
            g = np.dot(lattice, np.dot(nmat, inv_lat_vec))
            if np.all(abs(np.dot(g.T, g) - np.eye(3)) < threshold):
                flag_op = 1
                for bas in basis:
                    vec1 = bas - basis[0]
                    for i, j in enumerate(vec1):
                        if j < 0:
                            vec1[i] += 1.0
                    vec2 = np.dot(nmat, vec1)
                    for i, j in enumerate(vec2):
                        if j < 0:
                            vec2[i] += 1.0
                    if np.any(abs(vec1 - vec2) > threshold):
                        flag_op = 0
                        sgroup_ops.append(nmat)

                if flag_op:
                    pgroup_ops.append(nmat)
        return np.array(pgroup_ops), np.array(sgroup_ops) 
        

def apply_pg_site_dir(pg, site1, site2, threshold=1e-8):
        
    if np.any([np.all(abs(site1 - np.dot(g, site2)) < threshold) for g in pg]):
            return 1
    else:
            return 0

        
def apply_sg_site_dir(sg, site1, site2, threshold=1e-8):
        
    if np.any([np.all(abs(site1 - np.dot(g, site2)) < threshold) for g in sg]):
            return 1
    else:
            return 0

def apply_pg_trans(pg, trans1, trans2, threshold=1e-8):

    if np.any([np.all(abs(trans1[0] - np.dot(g, trans2[0])) < threshold) and np.all(abs(trans1[1] - np.dot(g, trans2[1])) < threshold) for g in pg]):
            return 1
    else:
            return 0

        
def apply_sg_trans(sg, trans1, trans2, threshold=1e-8):
        
    if np.any([np.all(abs(trans1[0] - np.dot(g, trans2[0])) < threshold)and np.all(abs(trans1[1] - np.dot(g, trans2[1])) < threshold) for g in sg]):
            return 1
    else:
            return 0
def read_data(data_file):
    site_data = []
    trans_data = []
    dict_loader = yaml.load(data_file)
    vacancy_data = dict_loader.get('vacancy_data')  # vacancy data  
    site_data = dict_loader.get('site_data')  # site data
    trans_data = dict_loader.get('trans_data')  # transition data
    for i, data in enumerate(trans_data):
        if len(data) == 6:
            trans_data[i].append([0, 0.0, 0.0])
    bulk = dict_loader.get('bulk')      
    for i in range(len(site_data)):
        site_data[i][2]-=bulk[1]
        site_data[i][1]=bulk[0]/site_data[i][1]
    for i in range(len(trans_data)):
        trans_data[i][3]-=bulk[1]
        trans_data[i][5]-=bulk[1]
        trans_data[i][2]=bulk[0]/trans_data[i][2]
        trans_data[i][4]=bulk[0]/trans_data[i][4]
        if trans_data[i][6][0]:
            trans_data[i][6][1]=bulk[0]/trans_data[i][6][1]
            trans_data[i][6][2]-=bulk[1]                        
    w0_data = []
    w1_data = []
    w2_data = []
    for data in trans_data:
        if np.all(data[0]==[0,0,0]):
            w0_data.append(data)
        elif np.all(data[1]==[0,0,0]):
            w2_data.append(data)
        else:
            w1_data.append(data)
    return vacancy_data, site_data, w0_data, w1_data, w2_data        

In [2]:
## get data
data = open('Sn_data.yaml', 'r')
vacancy_data, site_data, w0_data, w1_data, w2_data   = read_data(data)

In [3]:
## get list of deleted states and meta states
#for state in starset.states:
#    if state.i not in meta_sites and state.j in meta_sites:
#        if np.allclose(0.866025403784,np.linalg.norm(state.dx)):
#            to_del.append(state)
deleted_states = []
meta_states = []
new_w1_data = []
for jump in w1_data:
    if (jump[-1][0]):
        meta = np.array((np.array(jump[0])+np.array(jump[1]))/2).tolist()
        meta_states.append([meta,jump[-1][1],jump[-1][2]])
        new_jump1 = [jump[0],meta,jump[2],jump[3],jump[2],jump[3]]
        new_jump2 = [jump[1],meta,jump[4],jump[5],jump[4],jump[5]]
        new_w1_data.append(new_jump1)
        new_w1_data.append(new_jump2)
    else:
        deleted_states.append((np.array(jump[0])+np.array(jump[1]))/2)
        new_w1_data.append(jump)

In [4]:
deleted_states.append(np.array([0.5,0.5,0]))

In [5]:
a= 3.2342373809
c_a= 1.5989108537
c=a*c_a
HCP = crystal.Crystal.HCP(a0=a,c_a=c_a, chemistry="Zr")
pg,sg = build_point_group_ops(HCP.lattice/a, HCP.basis[0])
len(pg)
print(HCP)
meta_basis = HCP.Wyckoffpos(np.array([5/6,2/3,0.25]))
basis = HCP.basis[0] + meta_basis
HCPmeta = crystal.Crystal(HCP.lattice, basis[0:8], chemistry=["Zr"], noreduce=True)
sitelist = HCPmeta.sitelist(0)
vacancyjumps = HCPmeta.jumpnetwork(0, 1.01*a)
meta_sites = np.arange(2,8,1)
for pos,jlist in enumerate(vacancyjumps):
        if np.any([np.allclose(dx,a*np.array([0.5, -0.8660254, 0.])) for (i,j), dx in jlist]):
            ind1 = pos
            break
#print("ind1 = ",ind1)
for pos,jlist in enumerate(vacancyjumps):
        if np.any([np.allclose(dx,a*np.array([ 0.25, -0.4330127, 0.])) for (i,j), dx in jlist]):
            ind2 = pos
            break
#print("ind2 = ",ind2)
jumpnetwork = [vacancyjumps[1], vacancyjumps[ind2]]
jumpnetwork2 = [vacancyjumps[1], vacancyjumps[ind1]]

#Lattice:
  a1 = [ 1.61711869 -2.80093173  0.        ]
  a2 = [ 1.61711869  2.80093173  0.        ]
  a3 = [ 0.          0.          5.17125725]
#Basis:
  (Zr) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Zr) 0.1 = [ 0.66666667  0.33333333  0.75      ]


In [45]:
print(HCPmeta)

#Lattice:
  a1 = [ 1.61711869 -2.80093173  0.        ]
  a2 = [ 1.61711869  2.80093173  0.        ]
  a3 = [ 0.          0.          5.17125725]
#Basis:
  (Zr) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Zr) 0.1 = [ 0.66666667  0.33333333  0.75      ]
  (Zr) 0.2 = [ 0.16666667  0.83333333  0.75      ]
  (Zr) 0.3 = [ 0.83333333  0.16666667  0.25      ]
  (Zr) 0.4 = [ 0.83333333  0.66666667  0.25      ]
  (Zr) 0.5 = [ 0.66666667  0.83333333  0.75      ]
  (Zr) 0.6 = [ 0.16666667  0.33333333  0.75      ]
  (Zr) 0.7 = [ 0.33333333  0.16666667  0.25      ]


In [6]:
starset = stars.StarSetMeta(jumpnetwork, HCPmeta, 0, meta_sites = meta_sites)
starset.generate(4)
to_del = []
for i, state in enumerate(starset.states):
    if state.i not in meta_sites and state.j in meta_sites:
        if state.i==0: 
            if np.any([apply_pg_site_dir(pg,np.dot(HCP.invlatt,state.dx), site) for site in deleted_states]):
                to_del.append(state)
                #print(i)
        elif state.i==1: 
            if np.any([apply_sg_site_dir(sg,np.dot(HCP.invlatt,state.dx), site) for site in deleted_states]):
                to_del.append(state)                
                #print(i)

In [7]:
HCPdiffuser = OnsagerCalc.VacancyMediatedMeta(HCPmeta, 0, sitelist, jumpnetwork, 4, meta_sites = np.arange(2,8,1), jumpnetwork2= jumpnetwork2, deleted_states=to_del)

In [14]:
binding_entropy_list = []
binding_energy_list = []
for i,state in enumerate(HCPdiffuser.interactlist()):
        data_not_found = 1
        print(i,state)
        for site in site_data:
            if state.i==0: 
                if apply_pg_site_dir(pg,np.dot(HCP.invlatt,state.dx), site[0]):
                    print(site[0],site[2])
                    binding_entropy_list.append(site[1])
                    binding_energy_list.append(site[2])
                    data_not_found = 0
                    break
            else:        
                if apply_sg_site_dir(sg,np.dot(HCP.invlatt,state.dx), site[0]):
                    print(site[0],site[2])
                    binding_entropy_list.append(site[1])
                    binding_energy_list.append(site[2])
                    data_not_found = 0
                    break
        if data_not_found:
            for site in meta_states:
                if state.i==0: 
                    if apply_pg_site_dir(pg,np.dot(HCP.invlatt,state.dx), site[0]):
                        print(site[0],site[2])
                        binding_entropy_list.append(site[1])
                        binding_energy_list.append(site[2]-0.51727)
                        data_not_found = 0
                        break
                else:        
                    if apply_sg_site_dir(sg,np.dot(HCP.invlatt,state.dx), site[0]):
                        print(site[0],site[2])
                        binding_entropy_list.append(site[1])
                        binding_energy_list.append(site[2]-0.51727)
                        data_not_found = 0
                        break
        if data_not_found:           
                    print("no data; setting binding energy = 0")
                    binding_entropy_list.append(1.0)
                    binding_energy_list.append(0.0)

0 3.[0,0,0]:0.[1,0,0] (dx=[1.6171186904499997,-1.1102230246251565e-16,-0.0])
no data; setting binding energy = 0
1 1.[0,0,0]:2.[1,0,0] (dx=[0.8085593452250001,-1.4004658668643235,0.0])
no data; setting binding energy = 0
2 1.[0,0,0]:3.[0,0,0] (dx=[2.220446049250313e-16,-0.9336439112428823,-2.5856286258816352])
no data; setting binding energy = 0
3 0.[0,0,0]:6.[-1,0,0] (dx=[-2.425678035675,1.4004658668643235,0.0])
[0.5, 1.0, 0.0] 0.5828300000000581
4 1.[0,0,0]:7.[1,-1,0] (dx=[0.0,-2.8009317337286475,0.0])
[1.0, 0.5, 0.0] 0.4777199999999766
5 0.[0,0,0]:1.[0,0,0] (dx=[-0.0,-1.8672878224857652,2.5856286258816352])
[0.333333333, 0.666666667, -0.5] -0.06049999999993361
6 0.[0,0,0]:0.[-1,0,0] (dx=[-1.61711869045,2.8009317337286475,0.0])
[1.0, 1.0, 0.0] 0.023120000000062646
7 1.[0,0,0]:4.[0,-1,0] (dx=[-0.808559345225,-2.334109778107206,-2.5856286258816352])
no data; setting binding energy = 0
8 1.[0,0,0]:4.[1,0,0] (dx=[2.425678035675,-2.334109778107206,-2.5856286258816352])
no data; setting bi

In [15]:
##HCPtracer = {'preV': np.array([1.0,1.0/10000000.0]), 'eneV': np.array([0.0, 0.0]),
##             'preT0': np.array([ 1.0, 1.0]),
##             'eneT0': np.array([0, 0]),
##              }

# HCPtracer = {'preV': np.array([1.0,54.169024409/18.299152044],), 'eneV': np.array([0.0,0.51727]),
#              'preT0': np.array([ 54.169024409/9.26073917, 54.169024409/10.40702378]),
#              'eneT0': np.array([0.613339999999994,0.553549999999973]),
#               }

HCPsolute = {'preV': np.array([1.0,54.169024409/18.299152044],), 'eneV': np.array([0.0,0.51727]),
             'preT0': np.array([ 54.169024409/9.26073917, 54.169024409/10.40702378]),
             'eneT0': np.array([0.613339999999994,0.553549999999973]),
              }
HCPsolute['preSV'] = np.array(binding_entropy_list)
HCPsolute['eneSV'] = np.array(binding_energy_list)
HCPsolute['preS']= np.array([1.0,1.0])
HCPsolute['eneS']= np.array([0.,0])
# HCPtracer['preT2'] = np.array([ 54.169024409/9.26073917, 0.5* 54.169024409/10.40702378,54.169024409/10.40702378])
# HCPtracer['eneT2'] = np.array([0.613339999999994,0.553549999999973,1e12])


In [16]:
HCPsolute.update(HCPdiffuser.makeLIMBpreene(**HCPsolute))
for k,v in zip(HCPsolute.keys(), HCPsolute.values()): print(k,v)

eneT2 [ 0.55284  0.57667]
preS [ 1.  1.]
preT2 [ 6.14483534  5.54607392]
eneS [ 0.  0.]
preT0 [ 5.84931974  5.20504474]
preT1 [ 6.01761845  6.02611806  6.1885428   6.05738347  6.05876961  5.89142952
  5.869537    5.84749556  5.87942957  5.86819416  5.89008167  5.86819416
  5.86819416  5.84986229  5.82735422  5.83104683  5.87113685  5.87113685
  5.87113685  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.35212224  5.46801069  5.35334699  5.37285426  5.35267798  8.07101588
  8.07101588  5.3560698   5.23988871  5.22303517  5.25135212  5.23868992
  5.22184024  5.22184024  5.22184024  5.18549862  5.18878451  5.1887845

In [42]:
# omega1=HCPdiffuser.omegalist(1)[0]
# for i,j in enumerate(HCPdiffuser.mod_jumps1):
#     if j:
#         (s1,s2) = omega1[i]
#         print(s1.i,s1.j,np.linalg.norm(s1.dx),s1.dx)
#         print(s2.i,s2.j,np.linalg.norm(s2.dx),s2.dx)        
#         print()

            
# for i,j in enumerate(HCPdiffuser.zero_jumps1):
#     if j:
#         HCPsolute['eneT1'][i]=1e12     
# # for i,j in enumerate(HCPdiffuser.mod_jumps2):
# #     if j:
# #         HCPtracer['preT1'][i]*=0.5
# for i,j in enumerate(HCPdiffuser.zero_jumps2):
#     if j:
#         HCPsolute['eneT2'][i]=1e12                

In [15]:
# omega1=HCPdiffuser.omegalist(1)[0]
# for i,j in enumerate(omega1):
#     if(HCPdiffuser.om1_jt[i]):
#         (s1,s2) = j
#         print(i)
#         print(s1.i,s1.j,np.linalg.norm(s1.dx),s1.dx)
#         print(s2.i,s2.j,np.linalg.norm(s2.dx),s2.dx)        
#         print()            

In [11]:
for site in site_data:
    print(np.linalg.norm(HCP.lattice.dot(site[0])), site[0],HCP.lattice.dot(site[0]))

3.18939480254 [0.333333333, 0.666666667, -0.5] [ 1.61711869  0.93364391 -2.58562863]
3.2342373809 [1.0, 1.0, 0.0] [ 3.23423738  0.          0.        ]
4.54230455078 [1.333333333, 0.666666667, -0.5] [ 3.23423738 -1.86728782 -2.58562863]
5.17125725176 [0.0, 0.0, 1.0] [ 0.          0.          5.17125725]
5.57609380202 [0.333333333, 1.666666667, 0.5] [ 3.23423738  3.73457565  2.58562863]
5.60186346746 [-2.0, -1.0, 0.0] [-4.85135607  2.80093173  0.        ]
5.60186346746 [-1.0, -2.0, 0.0] [-4.85135607 -2.80093173  0.        ]
6.09936004839 [-1.0, 0.0, 1.0] [-1.61711869  2.80093173  5.17125725]


In [17]:
omega2=HCPdiffuser.omegalist(2)[0]
for j, (S1,S2) in enumerate(omega2):
    data_not_found = 1
    print(S1.i,S1.dx,S2.dx)
    if S1.i==0:
        for trans in w2_data:                        
            if apply_pg_site_dir(pg,np.dot(HCP.invlatt,S1.dx), trans[0]): 
                print(trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT2'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT2'][j]= trans[2]
                data_not_found = 0
                break
    else:
        for trans in w2_data:                        
            if apply_sg_site_dir(sg,np.dot(HCP.invlatt,S1.dx), trans[0]): 
                print(trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT2'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT2'][j]= trans[2]
                data_not_found = 0
                break
    if data_not_found:
            print("no data; limb used")
            #binding_entropy_list.append(1)
            #binding_energy_list.append(0.0)   



0 [-1.61711869  0.93364391 -2.58562863] [ 1.61711869 -0.93364391  2.58562863]
[0.33333333, 0.66666667, -0.5] [0.0, 0.0, 0.0] 3.5800080857693497 0.9316599999999653
1 [-1.61711869  2.80093173  0.        ] [ 1.61711869 -2.80093173  0.        ]
[1.0, 1.0, 0.0] [0.0, 0.0, 0.0] 3.516694958176717 0.7872100000000728


In [18]:
omega1=HCPdiffuser.omegalist(1)[0]
for j, (S1,S2) in enumerate(omega1):
    data_not_found = 1    
    if S1.i==0:
        for i,trans in enumerate(new_w1_data):                        
            if apply_pg_trans(pg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[0],trans[1]])):
                #print(S1.i,S1.dx,S2.dx)
                print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
            elif apply_pg_trans(pg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[1],trans[0]])):
                #print(S1.i,S1.dx,S2.dx)
                print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
    elif S1.i==1:
        for i,trans in enumerate(new_w1_data):                              
            if apply_sg_trans(sg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[0],trans[1]])):
                #print(S1.i,S1.dx,S2.dx)
                print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
            elif apply_sg_trans(sg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[1],trans[0]])):
                #print(S1.i,S1.dx,S2.dx)
                print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
    #if data_not_found:
            #print("no data; limb used")
            #if np.isclose(HCPsolute['eneT1'][j],0.55355):
            #    HCPsolute['preT1'][j]=5.20504474229
            #print(HCPsolute['eneT1'][j],HCPsolute['preT1'][j])
            #binding_entropy_list.append(1)
            #binding_energy_list.append(0.0)      

0 0 [0.33333333, 0.66666667, -0.5] [0.0, 1.0, -1.0] 5.281106353562441 0.6096200000000636
1 2 [0.33333333, 0.66666667, -0.5] [0.0, 0.0, -1.0] 8.796042370375247 0.7587399999999889
2 1 [0.33333333, 0.66666667, -0.5] [0.0, 1.0, 0.0] 3.4660213954334953 0.6066200000000208
3 7 [1.0, 1.0, 0.0] [1.33333333, 1.66666667, -0.5] 4.2429011733586774 0.6839700000000448
4 6 [1.0, 1.0, 0.0] [1.33333333, 0.66666667, -0.5] 4.915106342110049 0.6528900000000704
54 3 [0.33333333, 0.66666667, -0.5] [0.33333333, 1.66666667, -0.5] 4.966645350936026 0.6302600000000211
55 5 [0.33333333, 0.66666667, -0.5] [0.33333333, -0.33333333, -0.5] 3.2449123292073776 0.29921999999999116
56 4 [0.33333333, 0.66666667, -0.5] [-0.66666667, 0.66666667, -0.5] 4.313823502338216 0.5183000000000675
57 17 [1.0, 1.0, 0.0] [2.0, 2.0, 0.0] 5.437082045860324 0.6559600000000501
58 10 [1.0, 1.0, 0.0] [2.0, 1.0, 0.0] 4.4893747786672265 0.5071400000000494
59 11 [1.0, 1.0, 0.0] [1.0, 0.5, 0.0] 3.0917420669924764 0.4844799999999623
60 8 [1.0, 1.

In [19]:
for k,v in zip(HCPsolute.keys(), HCPsolute.values()): print(k,v)

eneT2 [ 0.93166  0.78721]
preS [ 1.  1.]
preT2 [ 3.58000809  3.51669496]
eneS [ 0.  0.]
preT0 [ 5.84931974  5.20504474]
preT1 [ 5.28110635  8.79604237  3.4660214   4.24290117  4.91510634  5.89142952
  5.869537    5.84749556  5.87942957  5.86819416  5.89008167  5.86819416
  5.86819416  5.84986229  5.82735422  5.83104683  5.87113685  5.87113685
  5.87113685  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974  5.84931974
  4.96664535  3.24491233  4.3138235   5.43708205  4.48937478  3.09174207
  3.44670119  5.05081778  5.20504474  5.22303517  5.84931974  5.20504474
  5.22184024  5.22184024  5.22184024  5.18549862  5.18878451  5.1887845

In [34]:
Temp=np.arange(500, 1010, 100)
#Temp = [500,1000]
#Temp = [500]
D_Onsager=[]
for T in Temp:
    pre=1e-8 # THz and Angstrom unit scaling
    Lvv, Lss, Lsv, L1vv = HCPdiffuser.Lij(*HCPdiffuser.preene2betafree(kB*T, **HCPsolute))
    #Lss=Lss*pre
    #D_Onsager.append([Lss[0,0],Lss[1,1],Lss[2,2]])    
    #print(T, Lsv[0,0]/Lss[0,0],Lsv[1,1]/Lss[1,1],Lsv[2,2]/Lss[2,2])
    print(T, Lsv[0,0]/Lss[0,0],Lsv[1,1]/Lss[1,1],Lsv[2,2]/Lss[2,2])
    Lss=Lss*pre
    Lvv=Lvv*pre
    #print(T, Lss[0,0],Lss[1,1],Lss[2,2])    
    #print(T, Lvv[0,0],Lvv[1,1],Lvv[2,2])
    
D_Onsager=np.array(D_Onsager)

500 -1.30874844479 -1.30874844479 -1.62044288475
600 -1.158386761 -1.158386761 -1.66532003836
700 -1.55437601083 -1.55437601083 -1.6922107782
800 -0.688501588464 -0.688501588464 -1.70914581815
900 -0.0785492237375 -0.0785492237375 -1.7203760396
1000 1.18736919159 1.18736919159 -1.72819014819
